In [1]:
import pandas as pd
dataset= pd.read_csv("messy_data.csv")
dataset

,Name,Email,Join Date,Subscription Fee,Country
0,Alice,alice@example.com,01-01-2023,$99.99,USA
1,Bob,bob@,2023/02/01,₹1200,India
2,Charlie,charlie@example.com,03-03-23,100,UK
3,David,NaN,"April 5, 2023",$89.99,USA
4,Eve,eve@example.com,NaN,€75,NaN
5,NaN,frank@sample,06-06-2023,$,India
6,Frank,frank@example.com,07/07/23,₹,India
7,Grace,grace@example,08-08-2023,99.9,France
8,Heidi,heidi@example.com,2023.09.09,NaN,Germany
9,Ivan,ivan@example.com,10-10-23,$100,USA


In [9]:
from Util import get_openai_api_key
OPEN_API_KEY=get_openai_api_key()

In [10]:
llm_config={
    "config_list":{
        "model":"gpt-3.5-turbo",
        "api_key":OPEN_API_KEY
    }
}

In [2]:
from autogen import AssistantAgent,UserProxyAgent,GroupChat,GroupChatManager

In [5]:
#agent1
user_agent=UserProxyAgent(
    name="user_agent",
    human_input_mode="NEVER",
    code_execution_config={
        "use_docker":False
    }
)

In [11]:
#agent 2
validator_agent=AssistantAgent(
    name="validator_agent",
    llm_config=llm_config,
    system_message="""
     You are a Data Validator.
    Given a DataFrame, scan for missing values and inconsistent formats like:
    - Empty strings or NaN
    - Invalid date formats (use dd-mm-yyyy)
    - Invalid currency formats (should be $ or ₹ followed by number)
    Output a JSON list: [{"row": <int>, "field": <str>, "issue": <str>}]
    """
)

In [14]:
#agent3
fixer=AssistantAgent(
    name="fixer",
    llm_config=llm_config,
    system_message="""
    You are a Data Fix Suggestion Agent.
    For each issue, propose a fix. Possible fixes:
    - Impute with mean/median/mode
    - Convert to valid date format
    - Strip or convert currency to float
    Output: [{"row": ..., "field": ..., "fix": "..."}]
    """
)

In [15]:
#agent 4
cleaner=AssistantAgent(
    name="cleaner",
    llm_config=llm_config,
    system_message=""" 
    You are a Data Cleaner.
    Apply the fixes suggested and return:
    - DataFrame head preview after cleaning
    - Save the full cleaned dataset as 'cleaned_output.csv' securely
    Do not expose raw data rows directly unless requested.
    """
)

In [23]:
csv_writer_agent = AssistantAgent(
    name="csv_writer_agent",
    system_message="""
You are a CSV writing assistant.

You will receive a filtered dataset (as text table or CSV string).
Task:
- Convert it into pandas DataFrame.
- Generate Python code to save it into 'verified_emails.csv'.
Output only Python code inside a code block.
""",
    llm_config=llm_config,
)

In [24]:
group_chat=GroupChat(
    agents=[
        validator_agent,
        fixer,
        cleaner,
        csv_writer_agent
    ],
    max_round=10,
    messages=[]
)
manager=GroupChatManager(
    groupchat=group_chat,
    llm_config=llm_config
)
response= user_agent.initiate_chat(
    manager,
    message=f""" Clean the following dataset:
    {dataset.head(10).to_string()}
    This is just the preview. Full dataset is available in memory.
    Here is the dataset of email messages:\n\n{dataset}\n\nPlease filter and convert the result to a CSV file."""
)


user_agent (to chat_manager):

 Clean the following dataset:
          Name                Email      Join Date Subscription Fee  Country
0    Alice    alice@example.com     01-01-2023           $99.99      USA
1      Bob                 bob@     2023/02/01            ₹1200    India
2  Charlie  charlie@example.com       03-03-23              100       UK
3    David                  NaN  April 5, 2023           $89.99      USA
4      Eve      eve@example.com            NaN              €75      NaN
5      NaN         frank@sample     06-06-2023                $    India
6    Frank    frank@example.com       07/07/23                ₹    India
7    Grace        grace@example     08-08-2023             99.9   France
8    Heidi    heidi@example.com     2023.09.09              NaN  Germany
9     Ivan     ivan@example.com       10-10-23             $100      USA
    This is just the preview. Full dataset is available in memory.
    Here is the dataset of email messages:

      Name           

In [25]:
data = {
    'Name': ['Alice', 'Bob', 'Charlie', 'David', 'Eve', 'Frank', 'Grace', 'Heidi', 'Ivan'],
    'Email': ['alice@example.com', 'bob@', 'charlie@example.com', 'imputed@email.com', 'frank@sample', 'frank@example.com', 'grace@example', 'heidi@example.com', 'ivan@example.com'],
    'Join Date': ['2023-01-01', '2023-02-01', '2023-03-03', '2023-04-05', '2023-06-06', '2023-07-07', '2023-08-08', '2023-09-09', '2023-10-10'],
    'Subscription Fee': [99.99, 'Imputed value', 100.0, 89.99, 'Imputed value', 'Imputed value', 99.9, 'Imputed value', 100.0],
    'Country': ['USA', 'India', 'UK', 'USA', 'Imputed value', 'India', 'France', 'Germany', 'USA']
}

df = pd.DataFrame(data)

Save the cleaned dataset to CSV
df.to_csv('verified_emails.csv', index=False)

SyntaxError: invalid syntax (1286929462.py, line 11)